In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm
from datetime import datetime

def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    delta = norm.cdf(d1)
    return call_price, delta

# 1. Setup e preço atual
ticker = 'SPY'
hoje = datetime.strptime('2025-07-07', '%Y-%m-%d')
r = 0.05  # taxa livre de risco anual

data = yf.download(ticker, period='60d', progress=False)['Close'].dropna()
S = float(data.iloc[-1])

# 2. Vencimento alvo ~30 dias
tck = yf.Ticker(ticker)
vencs = tck.options
vencs_dt = [datetime.strptime(x, '%Y-%m-%d') for x in vencs]
venc_alvo = min(vencs_dt, key=lambda d: abs((d - hoje).days - 30))
venc_alvo_str = venc_alvo.strftime('%Y-%m-%d')
T = (venc_alvo - hoje).days / 365

# 3. Filtra calls próximas ±5% preço atual
calls = tck.option_chain(venc_alvo_str).calls.dropna(subset=['strike', 'lastPrice', 'impliedVolatility'])
calls = calls[(calls['strike'] >= 0.95 * S) & (calls['strike'] <= 1.05 * S)].copy()

# 4. Volatilidade histórica (30d)
retornos = np.log(data / data.shift(1)).dropna()
hist_vol = float(retornos[-30:].std() * np.sqrt(252))

# 5. Função para classificar e calcular preço justo + delta
def classifica(row):
    iv = float(row['impliedVolatility'])
    preco_mercado = float(row['lastPrice'])
    K = float(row['strike'])
    
    preco_justo, delta = black_scholes_call(S, K, T, r, iv)
    
    preco_status = ("Cara" if preco_mercado > preco_justo * 1.1
                    else "Barata" if preco_mercado < preco_justo * 0.9
                    else "Justa")
    
    vol_status = ("IV > HV (otimista)" if iv > hist_vol * 1.1
                  else "IV < HV (calma)" if iv < hist_vol * 0.9
                  else "IV ≈ HV (neutra)")
    
    return pd.Series([preco_justo, delta, preco_status, vol_status])

# 6. Aplica função nas calls
calls[['PrecoJusto', 'Delta', 'PrecoStatus', 'VolStatus']] = calls.apply(classifica, axis=1)

# 7. Exibe opções caras e baratas (você pode filtrar por volume e openInterest depois)
print("\n=== Opções CARAS ===")
print(calls[calls['PrecoStatus'] == 'Cara'][['strike', 'lastPrice', 'PrecoJusto', 'Delta', 'impliedVolatility', 'VolStatus', 'volume', 'openInterest']])

print("\n=== Opções BARATAS ===")
print(calls[calls['PrecoStatus'] == 'Barata'][['strike', 'lastPrice', 'PrecoJusto', 'Delta', 'impliedVolatility', 'VolStatus', 'volume', 'openInterest']])


YF.download() has changed argument auto_adjust default to True


C:\Users\igorj\AppData\Local\Temp\ipykernel_34848\894554368.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S = float(data.iloc[-1])



=== Opções CARAS ===
    strike  lastPrice  PrecoJusto     Delta  impliedVolatility  \
21   598.0      33.91   30.546716  0.757895           0.207161   

             VolStatus  volume  openInterest  
21  IV > HV (otimista)     3.0             4  

=== Opções BARATAS ===
    strike  lastPrice  PrecoJusto     Delta  impliedVolatility  \
43   617.0      13.67   15.295063  0.588525           0.165231   
47   620.0      11.83   13.149291  0.550912           0.158303   
48   621.0      11.14   12.495876  0.537493           0.156518   
49   622.0      10.55   11.900047  0.523723           0.155343   
50   623.0      10.00   11.305447  0.509729           0.154000   
51   624.0       9.58   10.668756  0.495388           0.151894   
52   625.0       8.94   10.090996  0.480827           0.150399   
54   627.0       7.78    8.972960  0.450954           0.147317   
55   628.0       7.44    8.402583  0.435416           0.145303   
57   630.0       6.47    7.413039  0.404552           0.142709   
5

C:\Users\igorj\AppData\Local\Temp\ipykernel_34848\894554368.py:36: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  hist_vol = float(retornos[-30:].std() * np.sqrt(252))
